## Business Understanding 
Purpose: Ask relevant questions and define objectives for the problem that needs to be tackled

## Background
In recent years, the range of funding options for projects created by individuals and small companies has expanded considerably. In addition to savings, bank loans, friends & family funding and other traditional options, crowdfunding has become a popular and readily available alternative.

Kickstarter, founded in 2009, is one particularly well-known and popular crowdfunding platform. It has an all-or-nothing funding model, whereby a project is only funded if it meets its goal amount; otherwise no money is given by backers to a project. A huge variety of factors contribute to the success or failure of a project — in general, and also on Kickstarter. Some of these are able to be quantified or categorized, which allows for the construction of a model to attempt to predict whether a project will succeed or not. The aim of this project is to construct such a model and also to analyse Kickstarter project data more generally, in order to help potential project creators assess whether or not Kickstarter is a good funding option for them, and what their chances of success are.

### Final Deliverables


* Well designed presentation for non-technical stakeholders outlining findings and recommendations, as well as future work (10min presentation).
* Jupyter notebook following Data Science Lifecycle

### Things to think about

* Try different (at least 3) machine learning algorithms to check which performs best on the problem at hand
* What would be right performance metric: Precision, recall, accuracy, F1 score, or something else? (Check TPR?)
* Check for data imbalance


## Key Question 

We currently hold a task by Kickstarter to come up with a model to predict in a first step whether is project is likely to be successful, given certain project parameters. In a second step (out of scope), Kickstarter would like to be able to provide a good goal recommendation for creators( for example using staff picks etc.)

* Given certain project parameters, __is a campaign likely to succeed or fail?__ --> classification
* what would e a __reasonable goal reccomendation for creators__ --> regression



## Feature Glossary

Features included in model

* Target : state
*
*
*

## Dataset Description

- **backers_count**: Amount of people who backed this project
- **category**: 
- **country**: Country the project owner lives in
- **created_at**: Date when the prjoect was created
- **currency**: Currency of the country where the owner lives in
- **currency_trailing_code**: 
- **current_currency**: 
- **deadline**: Date until the project can be backed
- **disable_communication**: If the communication with owner was disabled or not
- **fx_rate**: Foreign exchange rate
- **goal**: Project is only funded when the goal amount is reached
- **launched_at**: Date when the project was launced
- **spotlight**: Highlighted projects (available to all projects that are successfully funded)
- **staff_pick**: Promissing project picked by Kickstarter employees
- **state**: Project status
- **state_changed_at**: Date when state changed the last time
- **static_usd_rate**: static USD Convergen rate at time
- **usd_pledged**: pledge amount converted to USD using Static_usd_rate


## Dataset New/Added Feature Description

- **campaign_days**: Days the Project was live
- **pledged_over**: Amount Pledged surpassing the Goal(with converted pledge amount) 
- **pre_launched_days**: Days before the Project was launched


## Target Metric

* F1 score — Since creators wouldn’t want the model to predict too many success that will turn out to be a failure (minimize False Positives) and backers would want to make sure the model capture as many success as possible (minimize False Negatives), I want a balance between precision and recall

## Outcome / Reccomendations
*
*
*

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.gridspec import GridSpec
import scipy as sc
from scipy.stats import kstest
import seaborn as sns
import math
import warnings
warnings.filterwarnings("ignore")

#Data mining
import os, glob

#Preprocessing
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import imblearn
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline, make_pipeline

## Predictive Modelling : Classification ("state")
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
#from xgboost import XGBClassifier

## Dashboard
Purpose : Define global variables and visuals

In [ ]:
random_state = 100
test_size = 0.3
sns.set(style = "ticks")
scoring = "f1"

## Data Mining

In [ ]:
# Import multiple Kickstarter csv files and merge into one dataframe

path = "data-2"
all_files = glob.glob(os.path.join(path, "*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=',')
    df['file'] = f.split('/')[-1]
    all_df.append(df)
    
merged_df = pd.concat(all_df, ignore_index=True, sort=True)

In [ ]:
#
# merged_df = pd.read_csv('data-2/Kickstarter_all.csv') ### brauche ich wenn ich den Anderen Kram nicht laufen lassen will

## Inspection and Data Cleaning

In [ ]:
merged_df.info()

In [ ]:
#save the merged data as .zip
#compression_opts = dict(method='zip', archive_name='out.csv')  
#merged_df.to_csv('out.zip', index=False, compression=compression_opts)

In [ ]:
# Display shape of "data"
merged_df.shape

In [ ]:
merged_df.head()

In [ ]:
merged_df.columns

In [ ]:
merged_df.groupby('state').count()

In [ ]:
pd.isnull(merged_df).sum()

## Data Handling

In [ ]:
# create a dataset for Inspection
final = merged_df.copy()

### Dropping Data

In [ ]:
drop_list = []

#### Dropping features with missing values

In [ ]:
drop_missing_values = ['blurb', 'friends', 'is_backing', 'is_starred', 'permissions', 'usd_type', 'location']
drop_list.extend(drop_missing_values)
final = final.drop(drop_missing_values, axis = 1)


#### Dropping useless features 

In [ ]:
drop_useless_features = ['creator', 'currency_symbol', 'name', 'photo', 'profile', 'slug', 'source_url', 'urls', 'file']
drop_list.extend(drop_useless_features)
final = final.drop(drop_useless_features, axis = 1)

#### Dropping redundant features

In [ ]:
drop_redundant_features = ['pledged', 'usd_pledged']
drop_list.extend(drop_redundant_features)
final = final.drop(drop_redundant_features, axis = 1)

In [ ]:
drop_list

#### Replacing features

In [ ]:
def clean_category(DataFrame): 
    cat_list = []
    subcat_list = []
    for e in DataFrame.category:
        string_list = e.split(',')
        if '/' in string_list[2]:
            cat_list.append(string_list[2].split('/')[0][8:])
            subcat_list.append(string_list[2].split('/')[1][:-1])
        else:
            cat_list.append(string_list[2][8:-1])
            subcat_list.append('None')
    DataFrame['category'] = cat_list
    DataFrame['sub_category'] = subcat_list
    return DataFrame

In [ ]:
modified_list = ['category','state']

In [ ]:
final = clean_category(final)

In [ ]:
final.category.unique()

In [ ]:
#replace successful and failed with 1 and 0
final.state.replace(['successful','failed'], [1,0],inplace=True)
final = final.query('state == [1,0]')
final.state = final.state.astype(int)
#

final.is_starrable = final.is_starrable.astype(int)
final.disable_communication = final.disable_communication.astype(int)
final.currency_trailing_code = final.currency_trailing_code.astype(int)
final.staff_pick = final.staff_pick.astype(int)
final.spotlight = final.spotlight.astype(int)
#drop live,susspended,cancelled
#final = final[final['state'] == [1,0]]


### Time conversions



In [ ]:
modified_list.extend(['launched_at', 'deadline', 'created_at', 'state_changed_at'])         

In [ ]:
#converting unix time 
final.launched_at = pd.to_datetime(final.launched_at,unit='s',infer_datetime_format=True)
final.deadline = pd.to_datetime(final.deadline,unit='s',infer_datetime_format=True)
final.created_at = pd.to_datetime(final.created_at,unit='s',infer_datetime_format=True)
final.state_changed_at = pd.to_datetime(final.state_changed_at,unit='s',infer_datetime_format=True)

### Writing df changes

In [ ]:
feature_list = list(merged_df.columns)

df_features = pd.DataFrame(feature_list,columns =['features'])
df_features['dropped'] = df_features.features.isin(drop_list)
df_features['drop_reason'] = ['missing_values' if x in drop_missing_values \
                              else 'useless' if x in drop_useless_features \
                              else 'redundant' if x in drop_redundant_features \
                              else 'None' for x in df_features['features']]
df_features['modified'] = df_features.features.isin(modified_list)

In [ ]:
df_features

# Data Exploration
Purpose: we gotta form a hypotheses / story about our defined problem by visually analyzing the data

In [ ]:
#new dataset for exploration
data_exp = final.copy()

In [ ]:
#years
final['launched_at_yr'] = [date.year for date in final['launched_at']]

In [ ]:
final.info()

In [ ]:
# Seperate continious vs. categorical variables
data_cat_col = ['category','country','sub_category','country','currency','current_currency','is_starrable','disable_communication','state']
data_cont_col = [x for x in final if x not in data_cat_col]
data_cat = final[data_cat_col]
data_cont = final[data_cont_col]

In [ ]:
# Check if scaling is needed ( we can do this by looking at the .skew()
final.skew()

In [ ]:
#Plot correlation heatmap for continious values
mask = np.triu(np.ones_like(data_cont.corr(), dtype=np.bool))
f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(data_cont.corr(), mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, fmt=".1g");


In [ ]:
#Plot correlation heatmap for continious values
mask = np.triu(np.ones_like(data_cont.corr(), dtype=np.bool))
f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(data_cont.corr(), mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, fmt=".1g");


In [ ]:
#Plot a histogram of our Target 'state' and see if it needs scaling for future work
data_exp['state'].value_counts(ascending=True).plot(kind='bar')

In [ ]:
plt.figure(figsize=(14,8))
total = float(len(data_exp))

ax = sns.countplot(x='category', hue='state',palette="ch:s=.25,rot=-.25", data=data_exp)
ax.set_xlabel("Category")
ax.set_ylabel(r"Projects")

for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_height()/total)
    x = p.get_x() + p.get_width()
    y = p.get_height()
    ax.annotate(percentage, (x, y),ha='right')
plt.show()

plt.figure(figsize=(14,10))

sns.countplot(x='category', hue='state', data=data_exp);

In [ ]:
plt.figure(figsize=(14,10))
sns.lineplot(x=data_exp.launched_at_yr, y=data_exp.converted_pledged_amount,data=data_exp);



In [ ]:
fig=plt.figure(figsize=(14,8))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()
#width = [4,2,3,1,5,3,5,7,8,7]
# changes here
ax1 = sns.histplot(x=data_exp.launched_at_yr, data=data_exp,color="black",ax=ax1)
sns.lineplot(x=data_exp.launched_at_yr, y=data_exp.converted_pledged_amount,data=data_exp,color="red", ax=ax2)

ax1.set_xlabel("Year")
ax1.set_ylabel(r"Projects")
ax2.set_ylabel(r"Pledged Amount")


# and here
#plt.xticks(data_exp.index, rotation=60)
plt.show()

# Feature Engineering

In [ ]:
new_features_list = ['pledged_over', 'campaign_days', 'pre_launched_days', 'launched_at_yr', 'goal_converted']

In [ ]:
#create new features
final['pledged_over'] = final.converted_pledged_amount - final.goal 
final['campaign_days'] = ((final.deadline - final.launched_at)/ np.timedelta64(1, 'h')).astype(int)
final['pre_launched_days'] = ((final.launched_at - final.created_at)/ np.timedelta64(1, 'h')).astype(int)

final['launched_at_yr'] = [date.year for date in final['launched_at']]

final['goal_converted'] = final["goal"] * final["static_usd_rate"]
#use log on goal_converted



In [ ]:
#use log on stuff
final['goal_converted_log'] = [(math.log(el)+1) for el in final['goal_converted']]
final['converted_pledged_amount_log'] = np.log(final['converted_pledged_amount']+1)
final['backers_count_log'] = np.log(final['backers_count']+1)

In [ ]:
final['goal_converted_log'].unique()

# Preprocessing (Train/Test Split and Basemodel)
In order to apply modelling on different dataset types, we should consider a nice way to do the splits.



In [ ]:
#define predictors and target variable X,y
#X = final.drop(["state"], axis=1)
#y = final["state"]

In [ ]:
final.info()

In [ ]:
# Get dummies for object variables: category, sub_category, currency, country

category_dummies = pd.get_dummies(final['category'], prefix='cat', drop_first=True) 
subcategory_dummies = pd.get_dummies(final['sub_category'], prefix='subcat', drop_first=True)
currency_dummies = pd.get_dummies(final['currency'], prefix='cur', drop_first=True) 
country_dummies = pd.get_dummies(final['country'], prefix='country', drop_first=True)
final = final.drop(['category', 'sub_category', 'currency', 'country'], axis=1) 
final = pd.concat([final, category_dummies, subcategory_dummies, currency_dummies, country_dummies], axis=1) 

In [ ]:
X = final.drop(["state", 'pledged_over', 'spotlight', 'currency_trailing_code', 'current_currency', 'created_at', 'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'state_changed_at', 'static_usd_rate'], axis=1)
y = final["state"]

In [ ]:
#Split data into training and testing sets
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=test_size,
                                                        random_state=random_state,
                                                        shuffle=True,
                                                   stratify=y)
# X_train = np.array(X_train)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)

In [ ]:
# create a dummy classifier model as Basemodel
dum_clf = DummyClassifier(strategy='constant',constant=1).fit(X_train,y_train)
y_pred_dum_clf = dum_clf.predict(X_test)

print(confusion_matrix(y_test,y_pred_dum_clf))
print(classification_report(y_test,y_pred_dum_clf))


In [ ]:
#for future work
#scores = cross_val_score(dummy_clf, X_train, y_train, scoring='f1', cv=10, n_jobs=-1)

In [ ]:
#use oversampling

# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority', random_state=random_state)
# fit and apply the transform
X_train_over, y_train_over = oversample.fit_resample(np.array(X_train), np.array(y_train))



In [ ]:
sum(y_train_over)

In [ ]:
len(y_train_over)

In [ ]:
features_scalable_list = ['backers_count',
 'converted_pledged_amount', 'goal_converted']

In [ ]:
data_cont_col

In [ ]:
X_train[features_scalable_list]

In [ ]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

In [ ]:
#use standard scaler on X_train and y_train
scaler = StandardScaler()
X_train_scaled[features_scalable_list] = scaler.fit_transform(np.array(X_train[features_scalable_list])) # Scaler is fitted to training data _only_
X_test_scaled[features_scalable_list] = scaler.transform(np.array(X_test[features_scalable_list])) # Already fitted scaler is applied to test data


#data_cat_col = ['category','country','sub_category','country','currency','current_currency','is_starrable','disable_communication']
#data_cont_col = [x for x in final if x not in data_cat_col]

In [ ]:
#use standard scaler on X_train_over and y_train_over

# Future Work

In [ ]:
# To do: save final df as csv
#compression_opts = dict(method='zip', archive_name='Kickstarter_all_clean.csv')  
#final.to_csv('Kickstarter_all_clean.zip', index=False, compression=compression_opts)

# Predictive Modelling
Purpose: Train machine learning models (supervised), evaluate their performance and use them to make predictions

* using f1 score as our metric

In [ ]:
#logistic regression

In [ ]:
#Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=100, 
                               random_state=random_state,
                               max_depth=5,
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1)




In [ ]:
def random_forest(X_train, Y_train, X_test, Y_test, rfc):
    rfc.fit(X_train_scaled, y_train)
    pred_train = rfc.predict(X_train_scaled)
    pred_test = rfc.predict(X_test_scaled)
    print(f1_score(Y_test, pred_test))
    print(classification_report(Y_test, pred_test)) 
    print(confusion_matrix(Y_test, pred_test))
    sns.heatmap(confusion_matrix(Y_test, pred_test), annot=True, cmap='YlGn', fmt='d');
    return pred_train, pred_test

In [ ]:
rfcmodel = random_forest(X_train, y_train, X_test, y_test, rfc)

### Defining hyperparameter space for grid-search -- reduce dataset? takes too long
param_grid = {'rf_clf__bootstrap': [True, False],
              'rf_clf__max_depth': [3, 6, 10, None],
              'rf_clf__max_features': ['auto', 'sqrt'],
              'rf_clf__min_samples_leaf': [1, 3],
              'rf_clf__min_samples_split': [2, 5],
              'rf_clf__n_estimators': [10, 50, 200]} # Others: kernel, degree (only for poly)
grid_rf_clf = GridSearchCV(rfc, param_grid, cv=5, scoring=scoring, 
                           verbose=4, n_jobs=-1)

grid_rf_clf.fit(X_train, y_train)

In [ ]:
#Support Vector Machines (use classifier)

In [ ]:
#AdaBoost
X_train.head()

In [ ]:
adaB = AdaBoostClassifier(base_estimator=None, n_estimators=5, learning_rate=1.0, random_state=1)
model = adaB.fit(X_train_scaled, y_train)
#X_train.info()

In [ ]:
def generic_clf(X_train, Y_train, X_test, Y_test, clf):
    clf.fit(X_train,Y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    print(f1_score(Y_test, pred_test))
    print(classification_report(Y_test, pred_test)) 
    print(confusion_matrix(Y_test, pred_test))
    sns.heatmap(confusion_matrix(Y_test, pred_test), annot=True, cmap='YlGn', fmt='d');
    return pred_train, pred_test

In [ ]:
adamodel = generic_clf(X_train, y_train, X_test, y_test, adaB)

In [ ]:
print(model.feature_importances_)

In [ ]:
X_train.info()

# Ensemble Methods

#use KNN,SVC,DTC,Randomforestclassifier,XGB....

# Define classifiers and run code
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    y_pred_pipe = pipe.predict(X_test)
    print("F1 Score: {:.2f}".format(f1_score(y_test, y_pred_pipe)))
    print("Confusion Matrix: \n" + str(confusion_matrix(y_test, y_pred_pipe)))
    print("\n")

# Future Work

In [ ]:
#use maybe RandomizedSearchCV on RandomForest or any given Algorithm

# Data Visualisation
Purpose: Communicate the findings with stakeholders using plots and interactive visualisations

# Findings 
Purpose: Summarize the key results and findings